In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [9]:
spotify= pd.read_csv('spotify_limpio_train.csv')

In [10]:
spotify.dtypes

Artist             object
Track              object
Album              object
Album_type         object
Stream            float64
Danceability      float64
Energy            float64
Key                 int64
Key_name           object
Loudness          float64
Speechiness       float64
Acousticness      float64
Liveness          float64
Valence           float64
Tempo             float64
Tempo_category     object
Duration_ms       float64
Licensed             bool
official_video       bool
dtype: object

In [11]:
spotify.head()

,Artist,Track,Album,Album_type,Stream,Danceability,Energy,Key,Key_name,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Tempo_category,Duration_ms,Licensed,official_video
0,MGMT,Me and Michael,Little Dark Age,album,51859353.0,0.628,0.841,9,A,-4.583,0.0268,0.0285,0.2540,0.706,111.985,Moderato / Allegretto,289853.0,True,True
1,Mc Lipi,Quebradas,Quebradas,single,31545318.0,0.762,0.714,9,A,-2.436,0.2190,0.1460,0.0415,0.558,112.985,Moderato / Allegretto,250588.0,True,True
2,Justin Quiles,La Esquina del Mall,La Esquina del Mall,single,17950239.0,0.701,0.628,6,F# / Gb,-5.026,0.2930,0.2430,0.3330,0.777,189.893,Presto,207388.0,True,True
3,Arctic Monkeys,Snap Out Of It,AM,album,376045969.0,0.728,0.638,5,F,-6.455,0.0336,0.2490,0.1160,0.872,130.014,Allegro,193030.0,True,True
4,MEDUZA,Tell Me Why - MEDUZA Remix,Tell Me Why (MEDUZA Remix),single,22769089.0,0.561,0.928,6,F# / Gb,-5.812,0.0371,0.0870,0.3320,0.182,125.988,Allegro,171429.0,True,True


In [12]:
spotify_sin_string= spotify.drop(['Artist', 'Track', 'Album', 'Album_type', 'Licensed', 'official_video', 'Tempo_category', 'Key_name'], axis=1)

## Modelo Baseline

Entrenamos regresión lineal utilizando las variables
disponibles sin aplicar transformaciones ni escalado. Los coeficientes obtenidos
presentan valores elevados, lo cual es esperable debido a la gran escala de la variable
objetivo (Stream) y de algunas variables explicativas.

In [13]:
X = spotify_sin_string.drop('Stream', axis=1)
y = spotify_sin_string['Stream']

model = LinearRegression()
model.fit(X, y)

y_pred_sklearn = model.predict(X)

In [14]:
print("\nParámetros (sklearn):")
print(f"w1 = {model.coef_[0]:.3f}, w2 = {model.coef_[1]:.3f}, b = {model.intercept_:.3f}")


Parámetros (sklearn):
w1 = 59799404.808, w2 = -153505404.006, b = 345623027.592


In [15]:
# Mostrar MSE final de ambos modelos
print("MSE (sklearn):", mean_squared_error(y, y_pred_sklearn))

MSE (sklearn): 5.824510501566578e+16


El error cuadrático medio (MSE) obtenido es muy alto, lo que se explica por el hecho de
que el error se eleva al cuadrado y la variable objetivo (stream) se mide en millones. Este resultado no se interpreta como un buen rendimiento, sino como una
referencia inicial que servirá para comparar modelos posteriores tras aplicar validación cruzada.

### Validación cruzada

El modelo baseline se evalúa mediante validación cruzada de 5 folds sobre el conjunto de
entrenamiento. Dado que el problema es de regresión, se utiliza el RMSE como métrica de
evaluación

In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    model,
    X,
    y,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

rmse_scores = -scores

print("RMSE por fold:", rmse_scores)
print("RMSE medio:", rmse_scores.mean())

RMSE por fold: [2.30165352e+08 2.54916432e+08 2.46372285e+08 2.38381716e+08
 2.36704312e+08]
RMSE medio: 241308019.3686923


Se obtiene un RMSE medio de
aproximadamente 244 millones de reproducciones. Esto implica que, de media, las
predicciones del modelo se desvían en ese orden de magnitud respecto al valor real.
Dado que la variable objetivo presenta valores muy elevados y una distribución muy
asimétrica, este resultado es esperable.

Usamos este valor como referencia para compararlo y ver si coseguimos reducirlo.

### Feature engineering

Probamos usando log(Stream) porque la popularidad musical tiene una distribución de cola larga,
y el log reduce la influencia de valores extremos y mejora el comportamiento del modelo.

No hay desequilibrio de clases, pero sí una distribución del target muy asimétrica,
típica de problemas de popularidad por lo que se aplican transformaciones como el logaritmo

Dado que el dataset no presenta una dimensión temporal,
no se aplican técnicas de agregación temporal ni lags.

In [17]:
import numpy as np

spotify_sin_string["log_stream"] = np.log1p(spotify_sin_string["Stream"])


In [ ]:
X = spotify_sin_string.drop(["Stream", "log_stream"], axis=1)
y = spotify_sin_string["log_stream"]

Función para evaluar un modelo con validación cruzada (RMSE)

In [19]:
def baseline_model(X, y, cv=5):

    model = LinearRegression()
    
    scores = cross_val_score(
        model,
        X,
        y,
        cv=cv,
        scoring="neg_root_mean_squared_error"
    )
    
    rmse_scores = -scores
    
    print("RMSE por fold:", rmse_scores)
    print("RMSE medio:", rmse_scores.mean())
    
    return rmse_scores.mean()


Normalizamos las variables
numéricas restantes que presentan escalas muy distintas, como Loudness, Tempo y
Duration_ms. Esta estandarización permite mejorar la estabilidad del modelo lineal y
evitar que variables con mayor rango dominen el proceso de entrenamiento.

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cols_to_scale = ["Loudness", "Tempo", "Duration_ms"]

spotify_sin_string[cols_to_scale] = scaler.fit_transform(
    spotify_sin_string[cols_to_scale]
)

In [21]:
baseline_model(X,y,5)

RMSE por fold: [1.58249521 1.59005025 1.57928975 1.62273151 1.59471818]
RMSE medio: 1.5938569791788275


np.float64(1.5938569791788275)

La normalización de las variables
explicativas no produce mejoras significativas en el error. Vamos a estudiar si las variables son muy relevantes

Analizamos la relevancia individual de
algunas variables.

In [22]:
X_sin_loudness = spotify_sin_string.drop(["Stream", "log_stream", 'Loudness'], axis=1)
y = spotify_sin_string["log_stream"]

In [23]:
baseline_model(X_sin_loudness,y,5)


RMSE por fold: [1.60435435 1.60786767 1.59533446 1.63792502 1.61889453]
RMSE medio: 1.6128752077384985


np.float64(1.6128752077384985)

Al eliminar *Loudness*, el RMSE
medio aumenta respecto al modelo base, lo que indica que esta variable aporta
información relevante al modelo. Por tanto, se decide mantener *Loudness* dentro del
conjunto final de variables.

In [24]:
X_sin_tempo = spotify_sin_string.drop(["Stream", "log_stream", 'Tempo'], axis=1)
y = spotify_sin_string["log_stream"]

In [25]:
baseline_model(X_sin_tempo,y,5)

RMSE por fold: [1.58279365 1.59022508 1.57893811 1.62327866 1.59469155]
RMSE medio: 1.5939854096422406


np.float64(1.5939854096422406)

Probamos eliminar *Tempo* y vemos que no aporta
ninguna mejora en el rendimiento, la mantenemos

ahora evaluamos el
impacto de eliminar simultáneamente las variables *Liveness* y *Duration_ms*

In [26]:
X_sin_liv_inst = spotify_sin_string.drop(["Stream", "log_stream", 'Liveness', 'Duration_ms'], axis=1)
y = spotify_sin_string["log_stream"]

In [27]:
baseline_model(X_sin_liv_inst,y,5)

RMSE por fold: [1.58451003 1.59159667 1.58089915 1.62314711 1.59440014]
RMSE medio: 1.5949106210991786


np.float64(1.5949106210991786)

El RMSE medio no mejora respecto al modelo base

In [28]:
X_sin_danc = spotify_sin_string.drop(["Stream", "log_stream", 'Danceability'], axis=1)
y = spotify_sin_string["log_stream"]

In [29]:
baseline_model(X_sin_danc,y,5)

RMSE por fold: [1.58180471 1.59102413 1.57893713 1.62374976 1.59511398]
RMSE medio: 1.5941259418490583


np.float64(1.5941259418490583)

Tampoco mejora al eliminar Danceability

se evalúa la eliminación
simultánea de *Speechiness* y *Acousticness*

In [30]:
X_sin_spee_aco = spotify_sin_string.drop(["Stream", "log_stream", 'Speechiness', 'Acousticness'], axis=1)
y = spotify_sin_string["log_stream"]

In [31]:
baseline_model(X_sin_spee_aco,y,5)

RMSE por fold: [1.59530148 1.61192673 1.59237203 1.64129685 1.61524792]
RMSE medio: 1.6112290030484142


np.float64(1.6112290030484142)

El RMSE medio aumenta de forma más notable respecto al modelo base, lo que
indica que la exclusión de estas variables perjudica el rendimiento del modelo. Por
tanto, se consideran relevantes y se mantienen.

probamos eliminando la variable key

In [32]:
X_sin_key = spotify_sin_string.drop(["Stream", "log_stream","Key"], axis=1)
y = spotify_sin_string["log_stream"]

In [33]:
baseline_model(X_sin_key,y,5)

RMSE por fold: [1.58276461 1.59033941 1.57921366 1.62264578 1.59497995]
RMSE medio: 1.5939886813189166


np.float64(1.5939886813189166)

A lo largo del proceso iterativo y selección de variables, se han evaluado
distintas combinaciones eliminando tanto variables individuales como pares de variables.
En todos los casos analizados, la eliminación de variables conduce a un empeoramiento o
no mejora del RMSE obtenido mediante validación cruzada.

En consecuencia, **se concluye que todas las variables consideradas aportan información
relevante al modelo** y se decide mantener el conjunto completo de predictores para las
siguientes etapas de modelado. Este resultado sugiere que la mejora del rendimiento
deberá venir de modelos más complejos en lugar de una reducción adicional del conjunto
de variables.